## Clean Inference Outputs from Finetuned Models

This notebook does a simple cleanup of the responses from finetuned models. The original inference jsonl files is stored in `datasets/llm_outputs/finetuned_inference/raw/`, under the column `full_response`. But this contains headers and the prompt as well. This notebook:
1. reads original jsonl as a pandas df
2. applies a simple regex to the value in column `full_response`, to extract just the assistant part
3. stores result for each row in a new column titled `prediction`
4. Saves final dataframe to a new jsonl in `datasets/llm_outputs/finetuned_inference/processed/`
 
From the resulting jsonl, the following three columns can now be used for assessing quality of model output:
* `instruction`
* `output`
* `prediction`


In [ ]:
import pandas as pd
import re

def clean_response(text):
    pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>\s*\n\n(.*?)<\|eot_id\|>"
    match = re.search(pattern, text, flags=re.DOTALL)
    if match:
        extracted_text = match.group(1)
        return extracted_text
    else:
        return None

in_file = "../datasets/llm_outputs/finetuned_inference/raw/Meta-Llama-3.1-8B-Instruct-bnb-4bit_inference_oral_arg_test_901.jsonl"
out_file = "../datasets/llm_outputs/finetuned_inference/processed/Meta-Llama-3.1-8B-Instruct-bnb-4bit_inference_oral_arg_test_901.jsonl"

df = pd.read_json(in_file, lines=True)

df['prediction'] = df['full_response'].apply(clean_response)
df.to_json(out_file, orient="records", lines=True)